In [ ]:
!pip install tensorflow-gpu==1.15

     |████████████████████████████████| 411.5 MB 7.2 kB/s 
     |████████████████████████████████| 3.8 MB 62.3 MB/s 
     |████████████████████████████████| 503 kB 42.2 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=a77a4b125ecdd0acfc6afda219e4aa5474f0110afef532b5bb597f6d9b1c5308
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succe

In [ ]:
!pip install -U bert-serving-server bert-serving-client

     |████████████████████████████████| 61 kB 202 kB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=aadd10743d9d3d7eedf471bc56b5c1fdff00c5fa5d0de09732c72b75152f2704
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D

In [ ]:
import pandas as pd

import re
import string
import nltk
from nltk.corpus import stopwords

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Sharing Caring/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Sharing Caring/test.csv')

In [ ]:
train.text=train.text.apply(lambda x:x.lower() )
test.text=test.text.apply(lambda x:x.lower())
#removing square brackets
train.text=train.text.apply(lambda x:re.sub('\[.*?\]', '', x) )
test.text=test.text.apply(lambda x:re.sub('\[.*?\]', '', x) )
train.text=train.text.apply(lambda x:re.sub('<.*?>+', '', x) )
test.text=test.text.apply(lambda x:re.sub('<.*?>+', '', x) )
#removing hyperlink
train.text=train.text.apply(lambda x:re.sub('https?://\S+|www\.\S+', '', x) )
test.text=test.text.apply(lambda x:re.sub('https?://\S+|www\.\S+', '', x) )
#removing puncuation
train.text=train.text.apply(lambda x:re.sub('[%s]' % re.escape(string.punctuation), '', x) )
test.text=test.text.apply(lambda x:re.sub('[%s]' % re.escape(string.punctuation), '', x) )
train.text=train.text.apply(lambda x:re.sub('\n' , '', x) )
test.text=test.text.apply(lambda x:re.sub('\n', '', x) )
#remove words containing numbers
train.text=train.text.apply(lambda x:re.sub('\w*\d\w*' , '', x) )
test.text=test.text.apply(lambda x:re.sub('\w*\d\w*', '', x) )

In [ ]:
token = Tokenizer()
token.fit_on_texts(train['text'])

In [ ]:
vocab_size = len(token.word_index) + 1
vocab_size

16834

In [ ]:
encode = token.texts_to_sequences(train['text'])
x = pad_sequences(encode, maxlen = 40, padding='post')

In [ ]:
# encode = token.texts_to_sequences(train['text'])
# x = pad_sequences(encode, maxlen = 40, padding='post')

vec_size = 100
model = Sequential()
model.add(Embedding(vocab_size, vocab_size, input_length = 40))

model.add(Conv1D(32, 2, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(x, train['target'], epochs = 10)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 16834)         283383556 
_________________________________________________________________
conv1d (Conv1D)              (None, 39, 32)            1077408   
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 19, 32)            0         
_________________________________________________________________
dropout (Dropout)            (None, 19, 32)            0         
_________________________________________________________________
dense (Dense)                (None, 19, 32)            1056      
_______________________________________________________

In [ ]:
encode_test = token.texts_to_sequences(test['text'])
x_test = pad_sequences(encode_test, maxlen = 40, padding='post')
pred = model.predict_classes(x_test)

In [ ]:
pred.shape

(3263, 1)

In [ ]:
pred

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [ ]:
test['target'] = pred
test = test.drop(columns=['keyword', 'location', 'text'])
test.to_csv(r'test_dl.csv', index=False)

In [ ]:
test.to_csv(r'/content/drive/MyDrive/Sharing Caring/test_dl_1.csv', index=False)